# 量子变分求解器在量子化学计算中的应用


`Linux` `CPU` `全流程` `初级` `中级` `高级`

[![](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/_static/logo_source.png)](https://gitee.com/mindspore/mindquantum/blob/master/tutorials/vqe_for_quantum_chemistry.ipynb)

## 概述

量子化学，指的是运用量子力学的基本理论及方法，求解含时或定态薛定谔方程的数值解。在高性能计算机上进行量子化学模拟已成为研究材料的物理、化学性质的重要手段。然而，精确求解薛定谔方程具有指数级的复杂度，可模拟的化学体系规模严重受制于此。近年量子计算的发展为解决这个问题提供了一条可行的路，有望在量子计算机上实现多项式复杂度下对薛定谔方程的高精度求解。

[Peruzzo等人](https://doi.org/10.1038/ncomms5213)在2014年首次将量子变分求解器(Variational quantum eigensolver, VQE)结合[幺正耦合簇理论](https://doi.org/10.1016/S0009-2614(89)87372-5)用于量子化学的模拟中，实现了He-H<sup>+</sup>基态能量的求解。量子变分求解器是一个量子-经典混合算法，在基于量子算法的化学模拟中应用广泛，本教程将介绍使用量子变分求解器求解分子体系基态能量的方法。

本教程的主要内容包括如下几个部分：

1. 量子化学原理简介
2. 量子变分求解器的应用
3. 使用mindquantum实现高效自动求导的VQE模拟

## 环境准备

本教程需要安装以下环境：
- NumPy
- SciPy
- [mindquantum](https://gitee.com/mindspore/mindquantum)
- [mindspore](https://gitee.com/mindspore/mindspore)
- PySCF
- openfermion
- openfermionpyscf

导入本教程所依赖模块

In [ ]:
import numpy as np
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import mindquantum as mq
from mindquantum import Circuit, X, Hamiltonian
from mindquantum.circuit import generate_uccsd
from mindquantum.nn import generate_pqc_operator
import mindspore as ms
import mindspore.context as context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

## 量子化学计算方法

量子化学的核心问题在于求解薛定谔方程(Schrödinger Equation)。一般来说，求解含时薛定谔方程(Time-dependent Schrödinger Equation)较为复杂，故引入玻恩-奥本海默近似(Born-Oppenheimer approximation, BO approximation)。BO近似认为，原子核质量远大于电子、运动速度远低于电子，故可以将两者进行分离变量，单独讨论原子核或电子的运动，于是可得到如下不含时的电子运动方程，也称为定态薛定谔方程：

$$
\hat{H} |\Psi\rangle = E |\Psi\rangle
$$

其中$\hat{H}$包含以下三项：

$$
\hat{H} = \hat{K} _{e} + \hat{V} _{ee} + \hat{V} _{Ne}
$$

分别为电子动能、电子-电子势能和电子-核势能。

有多种数值算法可以求解定态薛定谔方程。本教程将介绍其中的一类：波函数方法。波函数方法直接求解给定分子哈密顿量的本征波函数和本征能量，目前有大量的开源软件包可实现，如[PySCF](http://pyscf.org/)等。此处从一个简单的例子：氢化锂分子开始，使用openfermion结合openfermionpyscf插件进行。首先定义分子结构：

In [2]:
dist = 1.5
geometry = [
    ["Li", [0.0, 0.0, 0.0 * dist]],
    ["H",  [0.0, 0.0, 1.0 * dist]],
]
basis = "sto3g"
spin = 0
print("Geometry: \n", geometry)

Geometry: 
 [['Li', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 1.5]]]


上面的代码定义了一个Li-H键长为1.5Å分子。使用STO-3G基组进行计算。接下来使用openfermionpyscf，调用PySCF进行HF、CCSD和FCI计算。这三种方法属于波函数方法，开始计算之前，先对这些方法作一个简单的介绍：

### 波函数方法

求解定态薛定谔方程的方法之一是[Hartree-Fock(HF)](https://doi.org/10.1098/rspa.1935.0085)方法，该方法在二十世纪三十年代左右由Hartree等人提出，是量子化学计算中的基本方法。HF方法引入了单行列式近似，即$N$-电子体系的波函数由一个行列式形式的波函数表示：

$$
| \Psi \rangle = | \psi_{1} \psi_{2} \psi_{3} \dots \psi_{N} \rangle
$$

其中$| \psi_{1} \psi_{2} \psi_{3} \dots \rangle$代表由一组自旋轨道波函数$\{ \pi_{i} \}$构成的N阶行列式。
自旋轨道波函数$\psi_{i}$可进一步用一组形式已知的基函数展开：

$$\psi_{i} = \phi_{i} \eta_{i}$$
$$\phi_{i} = \sum_{\mu}{C_{\mu i} \chi_{\mu}}$$

其中$\{\chi_{\mu}\}$被称为基函数，可以是高斯函数等。
该近似考虑了电子间的交换作用，但是忽略了电子间的关联作用，故在无法正确计算如解离能等性质。

HF方法的改进可以从波函数展开定理出发。波函数展开定理可以表述为，若$\{ \psi_{i} \}$是一组完备的自旋轨道波函数，则$N$-电子体系波函数可以由$\{ \psi_{i} \}$构成的行列式波函数精确展开：

$$
| \Psi \rangle = \sum^{\infty} _ {i_{1} < i_{2} < \dots < i_{N}} {C_{i_{1} i_{2} \dots i_{N}} | \psi_{i_{1}} \psi_{i_{2}} \dots \psi_{i_{N}} \rangle}
$$

由此可得到Configuration Interaction(CI)方法：

$$
| \Psi_{CI} \rangle = C_{0} | \Psi_{HF} \rangle + \sum^{a\rightarrow\infty} _{i\in occ\\\\a\not\in occ}{C^{a} _{i} | \Psi^{a} _{i} \rangle } + \sum^{ab\rightarrow\infty} _{ij\in occ\\\\ab\not\in occ}{C^{ab} _{ij} | \Psi^{ab} _{ij} \rangle } 
$$

上式中的$| \Psi^{a}_{i} \rangle + \dots$代表电子由轨道$i$激发到轨道$a$的单激发波函数，以此类推。只考虑单激发和双激发的CI被称为CISD，即Configuration Interaction with singles and doubles。将基态HF波函数一直到N激发波函数全部考虑在内的Configuration Interaction被称为Full Configuration Interaction(FCI)，FCI波函数是定态薛定谔方程在给定基函数下的精确解。

### 二次量子化

在二次量子化表述下，体系的哈密顿量具有如下形式：

$$
\hat{H} = \sum_{p, q}{h^{p} _ {q} E^{p} _ {q}} + \sum_{p, q, r, s}{\frac{1}{2} g^{pq} _ {rs} E^{pq} _ {rs} }
$$

其中$E^{p} _ {q}$和$E^{pq} _ {rs}$分别为：

$$
E^{p} _ {q} = a^{\dagger} _ {p} a_{q}
$$
$$
E^{pq} _ {rs} = a^{\dagger} _ {p} a^{\dagger} _ {q} a_{r} a_{s}
$$

$a^{\dagger} _ {p}$和$a _ {q}$分别为产生算符(Creation Operator)和湮灭算符(Annihilation Operator)。

使用二次量子化的表述方法，可以非常方便地表示激发态波函数：

$$
| \Psi^{abc\dots} _ {ijk\dots} \rangle = a^{\dagger} _ {a} a^{\dagger} _ {b} a^{\dagger} _ {c} \dots a_{i} a_{j} a_{k} \dots | \Psi \rangle
$$

CI方法的一个改进是耦合簇理论(Coupled-Cluster theory, CC)。CC引入指数化算符：

$$
| \Psi_{CC} \rangle = \exp{(\hat{T})} | \Psi_{HF} \rangle
$$

其中耦合簇算符$\hat{T}$为对激发算符的求和：

$$
\hat{T} = \sum_{p\not\in occ\\\\q\in occ}{\theta^{p} _ {q} E^{p} _ {q}} + \sum_{pq\not\in occ\\\\rs\in occ}{\theta^{pq} _ {rs} E^{pq} _ {rs}} + \dots
$$

其中$\theta$和CI方法中的$C$类似，是待求解的参数。由指数的泰勒展开易知，即使耦合簇算符$\hat{T}$中只包含低阶激发项，$\exp{(\hat{T})}$也可以隐含部分高阶激发，这也使得CC方法向FCI波函数收敛的速度要远快于CI，同样截断到K激发，如K=2，CCSD的精度会超过CISD。


<!--
一般而言，若一个方法可以达到化学精度，即由此方法计算的能量和FCI能量之间的差值小于1 kcal/mol，则认为这个方法具有良好的精度，截断到三激发的CCSD(T)在大部分情况下都能符合这个标准
-->

电子关联作用的效果是使得总能量降低，故HF得到的基态能量会略高于CCSD和FCI。另外，从上述理论不难发现，FCI的计算量远大于CCSD和HF。我们使用openfermion封装的MolecularData和openfermionpyscf封装的run_pyscf函数来进行演示：

In [3]:
molecule_of = MolecularData(
    geometry,
    basis,
    multiplicity=2 * spin + 1
)
molecule_of = run_pyscf(
    molecule_of,
    run_scf=1,
    run_ccsd=1,
    run_fci=1
)

print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
print("FCI energy: %20.16f Ha" % (molecule_of.fci_energy))

Hartree-Fock energy:  -7.8633576215351164 Ha
CCSD energy:  -7.8823529091527016 Ha
FCI energy:  -7.8823622867987249 Ha


在上面的例子中，我们运行了Hartree-Fock(HF)、CCSD、FCI进行总能量的计算。若对运行时间进行统计，会发现$T_{HF}<T_{CCSD}\ll T_{FCI}$，换成计算量更大的体系如乙烯分子等会更明显一些。此外，对于计算得到的总能量，有$E_{HF}>E_{CCSD}>E_{FCI}$。计算完成后，我们将结果保存到`molecule_file`文件（即`molecule_of.filename`）中：

In [ ]:
molecule_of.save()
molecule_file = molecule_of.filename
print(molecule_file)

量子化学计算的一大阻碍是计算量。随着体系大小（电子数、原子数）的增加，求解FCI波函数和基态能量的时间消耗大约以$2^{N}$增长，即使是较小的分子如乙烯分子等，进行FCI计算也并不容易。量子计算机的出现为此提供了一条可能的解决途径，已有的研究表明，量子计算机可以多项式的时间复杂度模拟哈密顿量的含时演化，在量子处理器上进行化学模拟相较于经典计算机有指数级的加速。本教程将介绍其中一类量子算法：量子变分求解器。

## 量子变分求解器

量子变分求解器(Variational Quantum Eigensolver, VQE)是一类量子-经典混合(Hybrid quantum-classical)算法，应用变分原理实现对基态波函数的求解。其中，变分参数的优化步在经典计算机上进行。

### 变分原理

变分原理可使用如下形式表述：

$$
E_{0} \le \frac{\langle \Psi_{t} | \hat{H} | \Psi_{t} \rangle}{\langle \Psi_{t} | \Psi_{t} \rangle}
$$

上式中的$| \Psi_{t} \rangle$代表试探波函数。变分原理表明，在满足一定的条件下，任意试探波函数得到的基态能量总是大于等于真实的基态能量。变分原理为求解分子基态薛定谔方程提供了一种方法：使用一个参数化的函数$f(\theta)$作为精确基态波函数的近似，通过优化参数$\theta$来逼近精确的基态能量。

### 初态制备

在二次量子化表述下，$N$-电子HF波函数也具有非常简洁的形式：

$$
| \Psi_{HF} \rangle = \prod^{i=0} _{N-1}{a^{\dagger} _{i}| 0 \rangle}
$$

上式搭建了一个由量子化学波函数到量子计算的桥梁：用$|0\rangle$代表非占据轨道，用$|1\rangle$代表电子占据的轨道，由此可以将$N$-电子HF波函数映射为由一串$M+N$个量子比特$| 00\dots 11\dots \rangle$，$M$代表非占据轨道的数量。

以下代码构造了对应于LiH分子的HF初态波函数。在Jordan-Wigner变换下，相当于将$N$个$\text{X}$门作用于$|000\dots\rangle$上。

In [5]:
hartreefock_wfn_circuit = Circuit([X.on(i) for i in range(molecule_of.n_electrons)])
print(hartreefock_wfn_circuit)

X(0)
X(1)
X(2)
X(3)


基于此，我们可以构造如下形式的试探波函数：

$$
| \Psi_{t} \rangle = U(\theta) | \Psi_{HF} \rangle
$$

其中$U(\theta)$代表一个可通过量子线路模拟的幺正变换，$| \Psi_{HF} \rangle$作为初态，可通过多个单比特$\text{X}$门来方便地制备。$U(\theta) | \Psi_{HF} \rangle$的具体形式也被称为波函数拟设。
### 波函数拟设

前文提到的耦合簇理论是一个非常高效的波函数拟设。在量子计算机上使用，需要作一些修改：

$$
| \Psi_{UCC} \rangle = \exp{(\hat{T} - \hat{T}^{\dagger})} | \Psi_{HF} \rangle
$$

UCC即幺正耦合簇(Unitary Coupled-Cluster theory)，$\hat{T}^{\dagger}$代表$\hat{T}$的厄米共轭。如此，$\exp{(\hat{T} - \hat{T}^{\dagger})}$即为幺正算符。[Peruzzo等人](https://doi.org/10.1038/ncomms5213)在2014年首次使用VQE结合UCCSD(Unitary coupled-cluster with singles and doubles)拟设进行了量子计算机上的化学模拟实验。值得注意的是幺正耦合簇默认了耦合簇算符中的参数$\{\theta\}$是实数。在分子体系中该假设不会有问题；在周期性体系中，[刘杰等人](https://doi.org/10.1021/acs.jctc.0c00881)的研究表明幺正耦合簇会因为忽略复数部分而造成误差。本教程暂时不讨论幺正耦合簇在周期性体系中的应用。

使用mindquantum的circuit模块中的相应函数可读取先前保存在`molecule_file`的计算结果，方便地构造UCCSD波函数拟设，以及其对应的量子线路：

In [6]:
ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, n_electrons = generate_uccsd(molecule_file, th=-1)

ccsd:-7.882352909152702.
fci:-7.882362286798725.


完整的量子线路包含HF初态+UCCSD拟设，如下代码所示：

In [7]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

===============================Circuit Summary===============================
|Total number of gates  : 12612.                                            |
|Parameter gates        : 640.                                              |
|with 44 parameters are : p41, p28, p38, p10, p13, p31, p16, p3, p4, p29... |
|Number qubit of circuit: 12                                                |
Number of parameters: 44


对于LiH分子而言，其UCCSD波函数拟设中包含44个变分参数。该线路总共的量子比特门数量为12612，总共需要12个量子比特进行模拟。

### VQE的一般流程

使用VQE进行分子基态求解的一般流程如下：

1. 制备HF初态：$| 00\dots11\dots \rangle$；
2. 定义波函数拟设，如UCCSD等；
3. 将波函数拟设转化为参数化的量子线路；
4. 初始化变分参数，如全设为0等；
5. 在量子计算机上多次测量得到分子哈密顿量在该套变分参数下的能量$E(\theta)$以及能量关于参数的导数$\{ {\partial E} / {\partial \theta_{i}} \}$
6. 在经典计算机上使用优化算法，如梯度下降、BFGS等更新变分参数；
7. 将新的变分参数传入量子线路中进行更新；
8. 重复步骤(5)到(7)，直到满足收敛标准；
9. 结束

在第5步中，求取能量关于参数的导数$\{ {\partial E} / {\partial \theta_{i}} \}$在量子计算机上可通过parameter-shift rule来进行，在模拟器中也可通过模拟parameter-shift rule或者有限差分法来计算，是个较为耗时的过程。mindquantum基于mindspore框架，提供了类似于机器学习的自动求导功能，可以对参数化的量子线路进行反向传播求导，如此在模拟中可以达到高效计算导数的效果。以下使用mindquantum构造带自动求导功能的参数化UCCSD量子线路：

In [8]:
molecule_pqc = generate_pqc_operator(
    [], ansatz_parameter_names, total_circuit, 
    Hamiltonian(hamiltonian_QubitOp))

通过将参数的具体数值传入`molecule_pqc`，即可得到对应于此变分参数的能量$E(\theta)=\langle \Psi_{UCC}(\theta) | \hat{H} | \Psi_{UCC}(\theta) \rangle$以及关于每个变分参数的导数。

接下来需要进行VQE优化的(5)~(7)步，即对参数化量子线路进行优化。我们可以借助mindspore框架，使用参数化量子线路算子`molecule_pqc`构造一个机器学习模型，然后通过类似于训练神经网络的方法来优化变分参数。在[附录](#appendix)中会演示使用SciPy进行优化。

In [9]:
class PQCNet(ms.nn.Cell):
    def __init__(self, pqc):
        super(PQCNet, self).__init__()
        self.pqc = pqc
        self.weight =  Parameter(initializer("Zeros",
            len(self.pqc.ansatz_params_names)),
            name="weight")
        self.encoder_data_dummy = ms.Tensor([[0]], 
            self.weight.dtype)

    def construct(self):
        energy, _, grads = self.pqc(self.encoder_data_dummy, self.weight)
        return energy

molecule_pqcnet = PQCNet(molecule_pqc)

此处使用`"Zeros"`关键字，将所有的变分参数初始化为0。使用CCSD（耦合簇理论）或者MP2（二阶多体微扰论）的计算结果也可以作为幺正耦合簇变分参数的初始值，此处暂不对此进行讨论。此时有$E(\vec{0})=\langle \Psi_{UCC}(\vec{0}) | \hat{H} | \Psi_{UCC}(\vec{0}) \rangle = E_{HF}$：

In [10]:
initial_energy = molecule_pqcnet()
print("Initial energy: %20.16f" % (initial_energy.asnumpy()))

Initial energy:  -7.8633575439453125


最后使用mindspore的Adam优化器进行优化，学习率设置为$1\times 10^{-2}$，优化终止标准设置为$\left.|\epsilon|\right. = \left.|E^{k+1} - E^{k}|\right. \le 1\times 10^{-8}$

In [11]:
optimizer = ms.nn.Adam(molecule_pqcnet.trainable_params(), learning_rate=1e-2)
train_pqcnet = ms.nn.TrainOneStepCell(molecule_pqcnet, optimizer)

eps = 1.e-8
energy_diff = eps * 1000
energy_last = initial_energy.asnumpy() + energy_diff
iter_idx = 0
while (abs(energy_diff) > eps):
    energy_i = train_pqcnet().asnumpy()
    print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
    energy_diff = energy_last - energy_i
    energy_last = energy_i
    iter_idx += 1

print("Optimized energy: %20.16f" % (energy_i))
print("Optimized amplitudes: \n", molecule_pqcnet.weight.asnumpy())

Step   0 energy  -7.8633575439453125
Step   1 energy  -7.8578791618347168
Step   2 energy  -7.8738608360290527
Step   3 energy  -7.8696637153625488
Step   4 energy  -7.8716387748718262
Step   5 energy  -7.8722548484802246
Step   6 energy  -7.8763413429260254
Step   7 energy  -7.8799986839294434
Step   8 energy  -7.8786134719848633
Step   9 energy  -7.8745274543762207
Step  10 energy  -7.8739027976989746
Step  11 energy  -7.8769011497497559
Step  12 energy  -7.8790454864501953
Step  13 energy  -7.8783979415893555
Step  14 energy  -7.8772087097167969
Step  15 energy  -7.8777532577514648
Step  16 energy  -7.8795628547668457
Step  17 energy  -7.8810067176818848
Step  18 energy  -7.8813152313232422
Step  19 energy  -7.8808994293212891
Step  20 energy  -7.8806061744689941
Step  21 energy  -7.8808226585388184
Step  22 energy  -7.8812174797058105
Step  23 energy  -7.8812923431396484
Step  24 energy  -7.8810596466064453
Step  25 energy  -7.8808951377868652
Step  26 energy  -7.8809628486633301
S

## 附录

<a id="appendix"></a>

目前流行的教程中，对于二次量子化的分子哈密顿量以及幺正耦合簇算符的具体细节介绍较为缺乏，给入门带来了一定的困难。本教程接下来的附录将对哈密顿量中的单、双电子积分$h^{p} _ {q}$和$g^{pq} _ {rs}$，以及耦合簇算符的具体计算和构造进行解释。以下将跳过openfermionpyscf，不使用其封装好的函数，而是直接从PySCF的计算结果出发，从头构造VQE计算所需要的所有物理量。

接着前文的内容，导入一些额外的依赖：

In [12]:
from functools import reduce

from scipy.sparse.linalg import eigsh
from scipy.optimize import minimize
import pyscf
from pyscf import scf, cc, fci, ao2mo
from mindquantum.ops import FermionOperator, QubitOperator
from mindquantum.hiqfermion.transforms import Transform
from mindquantum.utils import hermitian_conjugated, count_qubits, normal_ordered
from mindquantum.circuit import decompose_single_term_time_evolution

手动使用PySCF进行HF、CCSD和FCI计算：

In [13]:
molecule = pyscf.gto.M(
    atom=geometry,
    basis=basis,
    spin=spin
)
energy_nuc = molecule.energy_nuc()

mf = scf.RHF(molecule)
print("Running Restricted Hartree-Fock...")
mf.kernel()
print("Running Restricted CCSD...")
mf_cc = cc.RCCSD(mf)
mf_cc.kernel()
print("Running Full Configuration Interaction...")
mf_fci = fci.FCI(mf)
energy_FCI = mf_fci.kernel()[0]

energy_RHF = mf.e_tot
energy_RCCSD = mf_cc.e_tot

print("Hartree-Fock energy: %20.16f Ha" % (energy_RHF))
print("CCSD energy: %20.16f Ha" % (energy_RCCSD))
print("FCI energy: %20.16f Ha" % (energy_FCI))

Running Restricted Hartree-Fock...
converged SCF energy = -7.86335762153512
Running Restricted CCSD...
E(CCSD) = -7.882352909152702  E_corr = -0.01899528761758478
Running Full Configuration Interaction...
Hartree-Fock energy:  -7.8633576215351164 Ha
CCSD energy:  -7.8823529091527016 Ha
FCI energy:  -7.8823622867987213 Ha


所需要的单、双电子积分可以通过PySCF的ao2mo模块进行计算，详细内容可参考PySCF的Documentation相关章节：

In [14]:
n_orb = molecule.nao_nr()
hcore = mf.get_hcore()
mo_coeff = mf.mo_coeff
one_body_mo = reduce(np.dot, (mo_coeff.T, hcore, mo_coeff))
two_body_mo = ao2mo.restore(1, pyscf.ao2mo.get_mo_eri(
    molecule, mo_coeff, compact=False), 
    n_orb
)
one_body_int = np.zeros([n_orb * 2] * 2)
two_body_int = np.zeros([n_orb * 2] * 4)

for p in range(n_orb):
    for q in range(n_orb):
        one_body_int[2 * p][2 * q] = one_body_mo[p][q]
        one_body_int[2 * p + 1][2 * q + 1] = one_body_mo[p][q]
        for r in range(n_orb):
            for s in range(n_orb):
                two_body_int[2 * p][2 * q][2 * r][2 * s] = two_body_mo[p][s][q][r]
                two_body_int[2 * p + 1][2 * q + 1][2 * r + 1][2 * s + 1] = two_body_mo[p][s][q][r]
                two_body_int[2 * p + 1][2 * q][2 * r][2 * s + 1] = two_body_mo[p][s][q][r]
                two_body_int[2 * p][2 * q + 1][2 * r + 1][2 * s] = two_body_mo[p][s][q][r]

得到单、双电子积分后，根据公式：

$$
\hat{H} = \sum_{p, q}{h^{p} _ {q} E^{p} _ {q}} + \sum_{p, q, r, s}{\frac{1}{2} g^{pq} _ {rs} E^{pq} _ {rs} }
$$

得到分子哈密顿量$\hat{H}$。使用mindquantum的FermionOperator算符进行构造：

In [15]:
hamiltonian_fermOp_1 = FermionOperator()
hamiltonian_fermOp_2 = FermionOperator()

for p in range(n_orb * 2):
    for q in range(n_orb * 2):
        hamiltonian_fermOp_1 += FermionOperator(
            ((p, 1), (q, 0)),
            one_body_int[p][q]
        )
for p in range(n_orb * 2):
    for q in range(n_orb * 2):
        for r in range(n_orb * 2):
            for s in range(n_orb * 2):
                hamiltonian_fermOp_2 += FermionOperator(
                    ((p, 1), (q, 1), (r, 0), (s, 0)),
                    two_body_int[p][q][r][s] * 0.5
                )

hamiltonian_fermOp_1 = normal_ordered(hamiltonian_fermOp_1)
hamiltonian_fermOp_2 = normal_ordered(hamiltonian_fermOp_2)
hamiltonian_fermOp = hamiltonian_fermOp_1 + hamiltonian_fermOp_2
hamiltonian_fermOp += FermionOperator((), energy_nuc)

此处使用Jordan-Wigner变换将费米子算符映射为Qubit算符：

In [16]:
hamiltonian_QubitOp = Transform(hamiltonian_fermOp).jordan_wigner()
n_qubits = count_qubits(hamiltonian_QubitOp)

耦合簇算符的构造如下所示。耦合簇算符，尤其是高阶激发算符，有多种不同的公式，对应的变分参数的个数、量子比特门的数量也不相同。此处采用下文代码中参考文献里的Spin-adapted激发算符，该公式也是[ADAPT-VQE](https://doi.org/10.1038/s41467-019-10988-2)的[开源代码](https://github.com/mayhallgroup/adapt-vqe)中所使用的：

In [17]:
def Pij(i: int, j: int):
    ia = i * 2 + 0
    ib = i * 2 + 1
    ja = j * 2 + 0
    jb = j * 2 + 1
    term1 = FermionOperator(
        ((ja, 0), (ib, 0)),
        1.0
    )
    term2 = FermionOperator(
        ((ia, 0), (jb, 0)),
        1.0
    )
    return np.sqrt(0.5) * (term1 + term2)

def Pij_dagger(i: int, j: int):
    return hermitian_conjugated(Pij(i, j))

def Qij_plus(i: int, j: int):
    ia = i * 2 + 0
    ib = i * 2 + 1
    ja = j * 2 + 0
    jb = j * 2 + 1
    term = FermionOperator(
        ((ja, 0), (ia, 0)),
        1.0
    )
    return term

def Qij_minus(i: int, j: int):
    ia = i * 2 + 0
    ib = i * 2 + 1
    ja = j * 2 + 0
    jb = j * 2 + 1
    term = FermionOperator(
        ((jb, 0), (ib, 0)),
        1.0
    )
    return term

def Qij_0(i: int, j: int):
    ia = i * 2 + 0
    ib = i * 2 + 1
    ja = j * 2 + 0
    jb = j * 2 + 1
    term1 = FermionOperator(
        ((ja, 0), (ib, 0)), 
        1.0
    )
    term2 = FermionOperator(
        ((ia, 0), (jb, 0)), 
        1.0
    )
    return np.sqrt(0.5) * (term1 - term2)

def Qij_vec(i: int, j: int):
    return [Qij_plus(i, j), Qij_minus(i, j), Qij_0(i, j)]

def Qij_vec_dagger(i: int, j: int):
    return [hermitian_conjugated(i) for i in Qij_vec(i, j)]

def Qij_vec_inner(a: int, b: int, i: int, j: int):
    vec_dagger = Qij_vec_dagger(a, b)
    vec = Qij_vec(i, j)
    result = FermionOperator()
    for i in range(len(vec)):
        result += vec[i] * vec_dagger[i]
    return result

def spin_adapted_T1(i: int, j: int):
    """
    Spin-adapted single excitation operators.
    Reference:
        Scuseria, G. E. et al., J. Chem. Phys. 89, 7382 (1988)
    """
    ia = i * 2 + 0
    ib = i * 2 + 1
    ja = j * 2 + 0
    jb = j * 2 + 1
    term1 = FermionOperator(((ia, 1), (ja, 0)), 1.0)
    term2 = FermionOperator(((ib, 1), (jb, 0)), 1.0)
    return [term1 + term2]

def spin_adapted_T2(creation_list: list, annihilation_list: list):
    """
    Spin-adapted double excitation operators.
    Reference:
        Igor O. Sokolov et al., J. Chem. Phys. 152, 124107 (2020) 
        Ireneusz W. Bulik et al., J. Chem. Theory Comput. 11, 3171−3179 (2015)
        Scuseria, G. E. et al., J. Chem. Phys. 89, 7382 (1988)
    """
    p = creation_list[0]
    r = annihilation_list[0]
    q = creation_list[1]
    s = annihilation_list[1]
    tpqrs1 = Pij_dagger(p, q) * Pij(r, s)
    tpqrs2 = Qij_vec_inner(p, q, r, s)
    tpqrs_list = [tpqrs1, tpqrs2]
    return tpqrs_list

然后根据公式

$$
\hat{T} = \sum_{p\not\in occ\\\\q\in occ}{\theta^{p} _ {q} E^{p} _ {q}} + \sum_{pq\not\in occ\\\\rs\in occ}{\theta^{pq} _ {rs} E^{pq} _ {rs}} + \dots
$$

以及公式

$$
| \Psi_{UCC} \rangle = \exp{(\hat{T} - \hat{T}^{\dagger})} | \Psi_{HF} \rangle
$$

构造幺正耦合簇UCCSD所需要的单、双激发算符。同样地，最后使用Jordan-Wigner变换将构造好的费米子激发算符转化为Qubit算符：

In [18]:
n_orb_occ = sum(molecule.nelec) // 2
n_orb_vir = n_orb - n_orb_occ
occ_indices = [i for i in range(n_orb_occ)]
vir_indices = [i + n_orb_occ for i in range(n_orb_vir)]

T1_singles = []
T2_doubles = []
constant = 1.0
for p_idx in range(len(vir_indices)):
    p = vir_indices[p_idx]
    for q_idx in range(len(occ_indices)):
        q = occ_indices[q_idx]

        tpq_list = spin_adapted_T1(p, q)
        for idx in range(len(tpq_list)):
            tpq = tpq_list[idx]
            tpq = tpq - hermitian_conjugated(tpq)
            tpq = normal_ordered(tpq)
            if len(list(tpq.terms)) > 0:
                T1_singles.append(tpq)

for p_idx in range(len(vir_indices)):
    p = vir_indices[p_idx]
    for q_idx in range(p_idx, len(vir_indices)):
        q = vir_indices[q_idx]
        for r_idx in range(len(occ_indices)):
            r = occ_indices[r_idx]
            for s_idx in range(r_idx, len(occ_indices)):
                s = occ_indices[s_idx]

                tpqrs_list = spin_adapted_T2([p, q], [r, s])
                for idx in range(len(tpqrs_list)):
                    tpqrs = tpqrs_list[idx]
                    tpqrs = tpqrs - hermitian_conjugated(tpqrs)
                    tpqrs = normal_ordered(tpqrs)
                    if len(list(tpqrs.terms)) > 0:
                        T2_doubles.append(tpqrs)

uccsd_operator_pool_fermOp = T1_singles + T2_doubles
uccsd_operator_pool_QubitOp = [Transform(op).jordan_wigner() for op in uccsd_operator_pool_fermOp]
n_params = len(uccsd_operator_pool_fermOp)

Hartree-Fock初态波函数与前文类似，在Jordan-Wigner变换下可以通过$N$个$\text{X}$门制备：

In [19]:
occ_indices_spin = [i for i in range(n_orb_occ * 2)]
hartreefock_wfn_circuit = Circuit([X.on(i) for i in occ_indices_spin])

幺正耦合簇波函数拟设对应的量子线路无法直接根据公式得到，需要作Trotter-Suzuki分解。Trotter-Suzuki分解可用下式描述：

$$
e^{\hat{A} + \hat{B}}\approx(e^{\hat{A}/k}e^{\hat{B}/k})^{k}
$$

幺正耦合簇波函数拟设在该分解下可写成一系列指数算符的乘积：

$$
|\Psi_{UCC} \rangle=\lim_{N\rightarrow\infty}{\prod _ {k=1}^{N}{\prod _ {i}^{N_{i}}{e^{\frac{\theta_{i}}{N}\hat{\tau} _ {i}}|\Psi_{0} \rangle}}}
$$

其中$\hat{\tau} _ {i}$属于反厄米算符$\{ E^{p} _ {q} - E^{q} _ {p}, E^{pq} _ {rs} - E^{sr} _ {qp} \}$之一。

在Jordan-Wigner变换后，$\hat{\tau} _ {i}$被映射为一系列Pauli算符的直积的加和：

$$
\hat{\tau} _ {i} = \sum_{\mu}{c_{\mu} P_{\mu i}}
$$
$$
P_{\mu i} = \prod^{\otimes} _ {k}{\sigma_{k \mu i}}, \sigma_{k \mu i} \in \text{{X, Y, Z}}
$$

一串Pauli算符的直积的指数$\exp{(P_{\mu i})}$有等效的量子线路表示，此处调用mindquantum的`decompose_single_term_time_evolution`函数生成对应的线路：

In [20]:
ansatz_circuit = Circuit()
ansatz_parameter_names = []
for i in range(n_params):
    qubitOp = uccsd_operator_pool_QubitOp[i]
    ansatz_parameter_names.append("p" + str(i))
    for term, coeff in qubitOp.terms.items():
        trottered_qubitOp = QubitOperator(term, coeff)
        ansatz_circuit += decompose_single_term_time_evolution(
            trottered_qubitOp,
            {"p" + str(i): coeff.imag}
        )

和前文一样，总的波函数线路等于HF初态+UCC拟设：

In [21]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

===============================Circuit Summary===============================
|Total number of gates  : 16772.                                            |
|Parameter gates        : 832.                                              |
|with 44 parameters are : p41, p28, p38, p10, p13, p31, p16, p3, p4, p29... |
|Number qubit of circuit: 12                                                |
Number of parameters: 44


需要注意的是，虽然此时变分参数的总数也是44个，但是单个变分参数对应的费米子算符和前文有所不同，故线路中总的量子比特门的数目也不一样。读者也可尝试不同的文献中给出的不同的激发算符来构造幺正耦合簇拟设所需的线路，以探索各种形式的激发算符对线路深度、变分参数数量的影响。使用新的`total_circuit`重新构造参数化量子线路算子`molecule_pqc`：

In [ ]:
molecule_pqc = generate_pqc_operator(
    [], ansatz_parameter_names, total_circuit,
    Hamiltonian(hamiltonian_QubitOp))

此处我们定义一个函数`energy_objective`，该函数返回能量以及能量关于每个变分参数的导数。

In [23]:
def energy_objective(parameters: np.ndarray, 
                     pqc: mq.nn.pqc.PQC):
    params_zero = ms.Tensor([[0]], ms.float32)
    params_opt = ms.Tensor(parameters, ms.float32)
    energy, _, grad = pqc(params_zero, params_opt)
    return energy.asnumpy()[0, 0], grad.asnumpy()[0, 0]

使用`energy_objective`得到初始能量。同样地，此时使用全为0的变分参数作为初始值：

In [24]:
init_amplitudes = np.zeros(n_params)
initial_energy, grad = energy_objective(init_amplitudes, molecule_pqc)
print("Initial energy: %20.16f" % (initial_energy))

Initial energy:  -7.8633575439453125


最后，调用SciPy的minimize，使用BFGS优化器进行VQE优化模拟。不同的波函数拟设、不同的优化算法会对最终结果有一定的影响，但不会有过大的差别，一般会在小数点后三位以下。

In [25]:
result = minimize(energy_objective, 
                  init_amplitudes, 
                  args=(molecule_pqc),
                  method="BFGS",
                  jac=True,
                  options={
                      "disp":True
                  })
print("Optimized energy: %20.16f" % (result.fun))
print("Optimized amplitudes: \n", result.x)

         Current function value: -7.882353
         Iterations: 15
         Function evaluations: 98
         Gradient evaluations: 86
Optimized energy:  -7.8823528289794922
Optimized amplitudes: 
 [-2.22631022e-04 -3.55662712e-02 -6.57172984e-16  3.97093689e-15
 -5.33872718e-15  4.49781015e-15  6.39930343e-04 -3.04572518e-03
  1.90964239e-03  2.86849924e-04  1.60018334e-02  2.79161962e-14
 -8.25128341e-16  2.27834837e-16 -5.78023991e-16  9.87635287e-15
 -4.86620764e-16 -8.24096183e-16 -7.91120404e-16  8.74431219e-06
  6.76903853e-04 -9.24311014e-04 -5.46154055e-02  9.22750289e-04
  3.11477834e-03  1.42495172e-02 -2.14072217e-16  6.86482069e-18
  4.65684505e-19 -2.58642139e-19 -1.66390756e-15  6.09039261e-17
  2.24957290e-17  6.77074460e-16  9.03726787e-04  3.11791185e-03
  1.42543419e-02 -1.55147586e-14  4.46844092e-16  1.97836927e-15
  1.13411621e-15  4.25921631e-04  3.50493542e-04  5.40203928e-02]
